In [ ]:
import os

import pandas as pd
pd.set_option('display.float_format', str)


In [ ]:
df = pd.read_csv("combined_final.csv")
df.rename(columns={"language": "Language", "runtime": "Runtime", "energy_usage": "Energy Usage", "execution_time": "Execution Time", "memory_usage": "Memory Usage", "cpu_usage": "CPU Usage"}, inplace=True)
df_no_id = df.drop(labels="id", axis=1)
df_no_id['Execution Time'] /= 1000
df_no_id["Runtime"] = df_no_id["Runtime"].apply(lambda x : x.capitalize())
df_no_id["Language"] = df_no_id["Language"].apply(lambda x: "JavaScript" if x == "javascript" else x.capitalize())
df_no_id

In [ ]:
# RQ 1: only runtime wasmer and then 3 languages
def make_table(df : pd.DataFrame, runtime : str) -> pd.DataFrame:
    df_c = df[(df["Runtime"] == runtime) & (df_no_id["Language"] == "C")].describe()
    df_go = df[(df["Runtime"] == runtime) & (df_no_id["Language"] == "Go")].describe()
    df_rust = df[(df["Runtime"] == runtime) & (df_no_id["Language"] == "Rust")].describe()
    df_js = df[(df["Runtime"] == runtime) & (df_no_id["Language"] == "JavaScript")].describe()

    a = df_c.add_prefix("C_")
    b = df_rust.add_prefix("Rust_")
    c = df_go.add_prefix("Go_")
    d = df_js.add_prefix("JavaScript_")
    e = pd.concat([a,b,c,d], axis=1)
    e.columns = pd.MultiIndex.from_tuples([tuple(s.split("_")) for s in e.columns])

    return d

In [ ]:
def rq_13(df : pd.DataFrame, runtime : str, column : str) -> pd.DataFrame:
    df_c = df[(df["Runtime"] == runtime) & (df_no_id["Language"] == "C")].rename(columns={column: "C"}).describe()["C"]
    df_go = df[(df["Runtime"] == runtime) & (df_no_id["Language"] == "Go")].rename(columns={column: "Go"}).describe()["Go"]
    df_rust = df[(df["Runtime"] == runtime) & (df_no_id["Language"] == "Rust")].rename(columns={column: "Rust"}).describe()["Rust"]
    df_js = df[(df["Runtime"] == runtime) & (df_no_id["Language"] == "JavaScript")].rename(columns={column: "JavaScript"}).describe()["JavaScript"]

    d = pd.concat([df_c,df_go,df_rust, df_js], axis=1)

    return d

In [ ]:
def make_table_metric(df : pd.DataFrame, metric : str) -> pd.DataFrame:
    df_wasmer = rq_13(df, "Wasmer", metric)
    df_wasmtime = rq_13(df, "Wasmtime", metric)

    a = df_wasmer.add_prefix("Wasmer_")
    b = df_wasmtime.add_prefix("Wasmtime_")

    d = pd.concat([a,b], axis=1)
    d.columns = pd.MultiIndex.from_tuples([tuple(s.split("_")) for s in d.columns])
    return d

In [ ]:
keys = {"Energy Usage": "J", "Execution Time": "s", "Memory Usage": "\%"}
for (k,v) in keys.items():
    parameter_name = k
    base = f"results_hyp3_{parameter_name.replace(' ', '-')}"
    file_name = f"{base}.tex"
    label = f"tab:{base}"
    b = make_table_metric(df_no_id, parameter_name)
    caption = f"Descriptive statistics for RQ3 and RQ4 - {parameter_name} ({v})."
    b.to_latex(file_name, column_format=">{\\bfseries}c" + len(b.columns) * "c", multicolumn_format="c", caption=caption, label=label, float_format="%.3f", position="H")
    os.system(f"./limit_width.sh {file_name}")
    os.system(f"./add_comment.sh {file_name}")

In [ ]:
# per language table
keys = {"Energy Usage": "J", "Execution Time": "s", "Memory Usage": "\%"}
for (k, v) in keys.items():
    parameter_name = k
    base = f"results_hyp1_{parameter_name.replace(' ', '-')}"
    file_name = f"{base}.tex"
    label = f"tab:{base}"
    b = df_no_id[["Language", "Runtime", parameter_name]].groupby("Language").describe().T.droplevel(0)
    b = b[["C", "Go", "Rust", "JavaScript"]]
    caption = f"Descriptive statistics for HYP1 - {parameter_name} ({v})."
    b.to_latex(file_name ,column_format=">{\\bfseries}c" + len(b.columns) * "c", multicolumn_format="c", caption=caption, label=label, float_format="%.3f", position="H")
    os.system(f"./add_comment.sh {file_name}")

In [ ]:
# per language table
keys = {"Energy Usage": "J", "Execution Time": "s", "Memory Usage": "\%"}
for (k, v) in keys.items():
    parameter_name = k
    base = f"results_hyp2_{parameter_name.replace(' ', '-')}"
    file_name = f"{base}.tex"
    label = f"tab:{base}"
    b = df_no_id[["Language", "Runtime", parameter_name]].groupby("Runtime").describe().T.droplevel(0)
    caption = f"Descriptive statistics for HYP2 - {parameter_name} ({v})."
    b.to_latex(file_name, column_format=">{\\bfseries}c" + len(b.columns) * "c", multicolumn_format="c", caption=caption, label=label, float_format="%.3f", position="H")
    os.system(f"./add_comment.sh {file_name}")